### Load Libraries

In [1]:
import psycopg2, os
import pandas as pd
import random
from datetime import datetime, timedelta
from faker import Faker

### Connect to PostgreSQL

In [56]:
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="foodmart",
    user="postgres",
    password="123")

print('Connecting to the PostgreSQL database...')

Connecting to the PostgreSQL database...


In [57]:
# create a cursor
cur = conn.cursor()

### Create Tables

In [4]:
createCmd = """ 
BEGIN;

-- Department Table
CREATE TABLE department (
    dept_id SERIAL PRIMARY KEY,
    dept_name VARCHAR(100) NOT NULL
);

-- Position Table
CREATE TABLE position (
    position_id SERIAL PRIMARY KEY,
    position_name VARCHAR(100) NOT NULL
);

-- Vendors Table
CREATE TABLE vendors (
    vendor_id SERIAL PRIMARY KEY,
    vendor_name VARCHAR(20) UNIQUE NOT NULL,
    phone_number VARCHAR(12),
    email VARCHAR(50),
    website VARCHAR(100)
);

-- Customers Table
CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    phone_number VARCHAR(15),
    date_joined TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Store Table
CREATE TABLE store (
    store_id SERIAL PRIMARY KEY,
    store_name VARCHAR(100),
    state VARCHAR(20),
    city VARCHAR(50),
    street VARCHAR(50),
    zip VARCHAR(10)
);

-- Products Table
CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    dept_id INTEGER,
    vendor_id INTEGER,
    brand VARCHAR(50),
    product_name VARCHAR(100),
    CONSTRAINT fk_dept_products FOREIGN KEY (dept_id) REFERENCES department (dept_id),
    CONSTRAINT fk_vendor_products FOREIGN KEY (vendor_id) REFERENCES vendors (vendor_id)
);

-- Transactions Table
CREATE TABLE transactions (
    transaction_id SERIAL PRIMARY KEY,
    store_id INTEGER NOT NULL,
    transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    transaction_type INTEGER DEFAULT 1 NOT NULL,
    CONSTRAINT fk_store_transactions FOREIGN KEY (store_id) REFERENCES store (store_id)
);

-- Sales Table
CREATE TABLE sales (
    sale_id SERIAL PRIMARY KEY,
    transaction_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    CONSTRAINT fk_product_sales FOREIGN KEY (product_id) REFERENCES products (product_id),
    CONSTRAINT fk_transaction_sales FOREIGN KEY (transaction_id) REFERENCES transactions (transaction_id)
);

-- Employee Table
CREATE TABLE employee (
    employee_id SERIAL PRIMARY KEY,
    store_id INTEGER NOT NULL,
    dept_id INTEGER NOT NULL,
    position_id INTEGER NOT NULL,
    first_name VARCHAR(30) NOT NULL,
    last_name VARCHAR(30) NOT NULL,
    email VARCHAR(100) NOT NULL,
    phone_number VARCHAR(15) NOT NULL,
    state VARCHAR(20) NOT NULL,
    city VARCHAR(50) NOT NULL,
    street VARCHAR(100) NOT NULL,
    zip VARCHAR(10) NOT NULL,
    hire_date DATE NOT NULL,
    CONSTRAINT fk_dept FOREIGN KEY (dept_id) REFERENCES department (dept_id),
    CONSTRAINT fk_position FOREIGN KEY (position_id) REFERENCES position (position_id),
    CONSTRAINT fk_store FOREIGN KEY (store_id) REFERENCES store (store_id)
);

CREATE TABLE customer_reviews (
    review_id SERIAL PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    store_id INTEGER DEFAULT 0, -- Default to 0 when no store is reviewed
    product_id INTEGER DEFAULT 0, -- Default to 0 when no product is reviewed
    rating INTEGER NOT NULL CHECK (rating BETWEEN 1 AND 5), -- Ratings must be between 1 and 5
    review_text TEXT,
    review_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_customer_reviews FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
    CONSTRAINT fk_product_reviews FOREIGN KEY (product_id) REFERENCES products (product_id) ON DELETE SET DEFAULT,
    CONSTRAINT fk_store_reviews FOREIGN KEY (store_id) REFERENCES store (store_id) ON DELETE SET DEFAULT,
    CONSTRAINT check_review_type CHECK (
        (store_id <> 0 OR product_id <> 0) -- At least one of store or product must be reviewed
    )
);


-- Deliveries Table
CREATE TABLE deliveries (
    delivery_id SERIAL PRIMARY KEY,
    product_id INTEGER NOT NULL,
    vendor_id INTEGER NOT NULL,
    store_id INTEGER NOT NULL,
    delivery_date DATE NOT NULL,
    quantity INTEGER NOT NULL,
    CONSTRAINT fk_product_deliveries FOREIGN KEY (product_id) REFERENCES products (product_id),
    CONSTRAINT fk_store_deliveries FOREIGN KEY (store_id) REFERENCES store (store_id),
    CONSTRAINT fk_vendor_deliveries FOREIGN KEY (vendor_id) REFERENCES vendors (vendor_id)
);

-- Employee Schedule Table
CREATE TABLE employee_schedule (
    schedule_id SERIAL PRIMARY KEY,
    employee_id INTEGER NOT NULL,
    start_time TIMESTAMP,
    end_time TIMESTAMP,
    CONSTRAINT fk_employee_schedule FOREIGN KEY (employee_id) REFERENCES employee (employee_id) ON DELETE CASCADE
);

-- Employee Time Tracking Table
CREATE TABLE employee_time_tracking (
    time_tracking_id SERIAL PRIMARY KEY,
    employee_id INTEGER NOT NULL,
    start_time TIMESTAMP NOT NULL,
    end_time TIMESTAMP NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_employee_time_tracking FOREIGN KEY (employee_id) REFERENCES employee (employee_id)
);

-- Inventory Table
CREATE TABLE inventory (
    inventory_id SERIAL PRIMARY KEY,
    store_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER DEFAULT 0 NOT NULL,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_product_inventory FOREIGN KEY (product_id) REFERENCES products (product_id),
    CONSTRAINT fk_store_inventory FOREIGN KEY (store_id) REFERENCES store (store_id)
);

-- Payroll Table
CREATE TABLE payroll (
    payroll_id SERIAL PRIMARY KEY,
    employee_id INTEGER NOT NULL,
    pay_period_start DATE NOT NULL,
    pay_period_end DATE NOT NULL,
    hour_wage NUMERIC(10, 2) NOT NULL,
    CONSTRAINT fk_employee_payroll FOREIGN KEY (employee_id) REFERENCES employee (employee_id)
);

-- Price History Table
CREATE TABLE price_history (
    history_id SERIAL PRIMARY KEY,
    product_id INTEGER NOT NULL,
    store_id INTEGER,
    price NUMERIC(10, 2) NOT NULL,
    effective_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_product_price_history FOREIGN KEY (product_id) REFERENCES products (product_id),
    CONSTRAINT fk_store_price_history FOREIGN KEY (store_id) REFERENCES store (store_id)
);

-- Product Waste Table
CREATE TABLE product_waste (
    waste_id SERIAL PRIMARY KEY,
    product_id INTEGER NOT NULL,
    store_id INTEGER NOT NULL,
    dept_id INTEGER,
    quantity INTEGER NOT NULL,
    waste_reason VARCHAR(255) NOT NULL,
    waste_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_dept_waste FOREIGN KEY (dept_id) REFERENCES department (dept_id),
    CONSTRAINT fk_product_waste FOREIGN KEY (product_id) REFERENCES products (product_id),
    CONSTRAINT fk_store_waste FOREIGN KEY (store_id) REFERENCES store (store_id)
);

END;
            """

cur.execute(createCmd)
conn.commit()

### Create Trigger Statements

In [5]:
createCmd = """ 
-- Add a trigger to handle updating the `updated_at` column
CREATE OR REPLACE FUNCTION update_timestamp()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = CURRENT_TIMESTAMP;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER set_updated_at
BEFORE UPDATE ON employee_time_tracking
FOR EACH ROW
EXECUTE FUNCTION update_timestamp();


CREATE OR REPLACE FUNCTION update_inventory_on_delivery()
RETURNS TRIGGER AS $$
BEGIN
    -- Check if the inventory record already exists
    IF EXISTS (
        SELECT 1
        FROM inventory
        WHERE store_id = NEW.store_id AND product_id = NEW.product_id
    ) THEN
        -- Update the existing inventory record
        UPDATE inventory
        SET quantity = quantity + NEW.quantity,
            last_updated = CURRENT_TIMESTAMP
        WHERE store_id = NEW.store_id AND product_id = NEW.product_id;
    ELSE
        -- Insert a new inventory record
        INSERT INTO inventory (store_id, product_id, quantity, last_updated)
        VALUES (NEW.store_id, NEW.product_id, NEW.quantity, CURRENT_TIMESTAMP);
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_update_inventory
AFTER INSERT ON deliveries
FOR EACH ROW
EXECUTE FUNCTION update_inventory_on_delivery();


CREATE OR REPLACE FUNCTION update_inventory_on_sales()
RETURNS TRIGGER AS $$
BEGIN
    -- Check if the inventory record exists
    IF EXISTS (
        SELECT 1
        FROM inventory
        WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
          AND product_id = NEW.product_id
    ) THEN
        -- If it's a sale, decrease inventory quantity
        IF (SELECT transaction_type FROM transactions WHERE transaction_id = NEW.transaction_id) = 0 THEN
            UPDATE inventory
            SET quantity = quantity - NEW.quantity,
                last_updated = CURRENT_TIMESTAMP
            WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
              AND product_id = NEW.product_id;
        -- If it's a return, increase inventory quantity
        ELSE
            UPDATE inventory
            SET quantity = quantity + NEW.quantity,
                last_updated = CURRENT_TIMESTAMP
            WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
              AND product_id = NEW.product_id;
        END IF;
    ELSE
        -- Optionally insert a default inventory record for missing products
        INSERT INTO inventory (store_id, product_id, quantity, last_updated)
        VALUES (
            (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id),
            NEW.product_id, 0, CURRENT_TIMESTAMP
        );
        
        -- Proceed with the inventory update
        IF (SELECT transaction_type FROM transactions WHERE transaction_id = NEW.transaction_id) = 0 THEN
            UPDATE inventory
            SET quantity = quantity - NEW.quantity,
                last_updated = CURRENT_TIMESTAMP
            WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
              AND product_id = NEW.product_id;
        ELSE
            UPDATE inventory
            SET quantity = quantity + NEW.quantity,
                last_updated = CURRENT_TIMESTAMP
            WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
              AND product_id = NEW.product_id;
        END IF;
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;


CREATE TRIGGER trigger_update_inventory_sales
AFTER INSERT ON sales
FOR EACH ROW
EXECUTE FUNCTION update_inventory_on_sales();

CREATE OR REPLACE FUNCTION update_inventory_on_waste()
RETURNS TRIGGER AS $$
BEGIN
    -- Check if the inventory record exists
    IF EXISTS (
        SELECT 1
        FROM inventory
        WHERE store_id = NEW.store_id AND product_id = NEW.product_id
    ) THEN
        -- Update the inventory record by subtracting the waste quantity
        UPDATE inventory
        SET quantity = quantity - NEW.quantity,
            last_updated = CURRENT_TIMESTAMP
        WHERE store_id = NEW.store_id AND product_id = NEW.product_id;

        -- Ensure inventory quantity does not drop below zero
        IF (SELECT quantity FROM inventory WHERE store_id = NEW.store_id AND product_id = NEW.product_id) < 0 THEN
            RAISE EXCEPTION 'Inventory quantity cannot be negative for store_id % and product_id %',
            NEW.store_id, NEW.product_id;
        END IF;
    ELSE
        -- If no inventory record exists, handle gracefully
        RAISE EXCEPTION 'Inventory record not found for store_id % and product_id %',
            NEW.store_id, NEW.product_id;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_update_inventory_waste
AFTER INSERT ON product_waste
FOR EACH ROW
EXECUTE FUNCTION update_inventory_on_waste();


            """

cur.execute(createCmd)
conn.commit()

### Create Sample Date

### Department

As part of the ETL process departments can be edited only as an owner of the website. This would be an admin view allowing an owner to change departments

In [6]:
def create_department_list():
    """
    Creates a list of grocery store departments with their IDs and names
    and returns it as a Pandas DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing department IDs and names.
    """
    # Example department data
    departments = [
        {"dept_id": 1, "dept_name": "Produce"},
        {"dept_id": 2, "dept_name": "Dairy"},
        {"dept_id": 3, "dept_name": "Bakery"},
        {"dept_id": 4, "dept_name": "Frozen Foods"},
        {"dept_id": 5, "dept_name": "Meat & Seafood"},
        {"dept_id": 6, "dept_name": "Pantry"},
        {"dept_id": 7, "dept_name": "Beverages"},
        {"dept_id": 8, "dept_name": "Household Supplies"},
        {"dept_id": 9, "dept_name": "Health & Beauty"},
        {"dept_id": 10, "dept_name": "Snacks"}
    ]

    # Create DataFrame
    df = pd.DataFrame(departments)
    
    return df

# Usage
department_df = create_department_list()
department_df

,dept_id,dept_name
0,1,Produce
1,2,Dairy
2,3,Bakery
3,4,Frozen Foods
4,5,Meat & Seafood
5,6,Pantry
6,7,Beverages
7,8,Household Supplies
8,9,Health & Beauty
9,10,Snacks


In [7]:
# Define the table and column structure
table_name = "department"
columns = ", ".join(department_df.columns)
placeholders = ", ".join(["%s"] * len(department_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in department_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the department table.


### Positions

As part of the ETL process positions can be edited only as an owner of the website. This would be an admin view allowing an owner to change positions.

In [8]:
# Function to generate position data and return a DataFrame
def generate_position_data(num_records=10):
    """
    Generates position data and returns it as a DataFrame.
    
    Args:
        num_records (int): Number of records to generate.

    Returns:
        pd.DataFrame: A DataFrame containing position data.
    """
    # List of typical food store job positions
    position_names = [
        "Cashier", "Stock Clerk", "Store Manager", "Assistant Manager", 
        "Customer Service Representative", "Bakery Staff", "Deli Clerk", 
        "Meat Cutter", "Produce Clerk", "Sales Associate", "Grocery Bagger", 
        "Inventory Manager"
    ]
    
    # Generate data for the DataFrame
    data = []
    for position_id in range(1, num_records + 1):
        position_name = position_names[position_id % len(position_names)]  # Loop over position names
        data.append({"position_id": position_id, "position_name": position_name})
    
    # Create a DataFrame
    df = pd.DataFrame(data)
    return df

# Generate 12 position records
position_df = generate_position_data(12)

# Display the DataFrame
position_df

,position_id,position_name
0,1,Stock Clerk
1,2,Store Manager
2,3,Assistant Manager
3,4,Customer Service Representative
4,5,Bakery Staff
5,6,Deli Clerk
6,7,Meat Cutter
7,8,Produce Clerk
8,9,Sales Associate
9,10,Grocery Bagger


In [9]:
# Define the table and column structure
table_name = "position"
columns = ", ".join(position_df.columns)
placeholders = ", ".join(["%s"] * len(position_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in position_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the position table.


### Stores

As part of the ETL process stores can be edited only as an owner of the website. This would be an admin view allowing an owner to change stores as stores are added.

In [10]:
# Function to create a DataFrame for store data
def generate_store_data():
    """
    Generates store data for three stores located in Queens and Brooklyn 
    and returns it as a DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing store data.
    """
    # Store data for three specific stores
    stores = [
        {
            "store_id": 1,
            "store_name": "ABC Food Mart Queens 1",
            "state": "NY",
            "city": "Queens",
            "street": "Main Street",
            "zip": "11354"
        },
        {
            "store_id": 2,
            "store_name": "ABC Food Mart Queens 2",
            "state": "NY",
            "city": "Queens",
            "street": "Broadway",
            "zip": "11372"
        },
        {
            "store_id": 3,
            "store_name": "ABC Food Mart Brooklyn 1",
            "state": "NY",
            "city": "Brooklyn",
            "street": "Flatbush Avenue",
            "zip": "11226"
        },
          {
            "store_id": 4,
            "store_name": "ABC Food Mart Brooklyn 2",
            "state": "NY",
            "city": "Brooklyn",
            "street": "Fulton Street",
            "zip": "11201"
        },
          {
            "store_id": 5,
            "store_name": "ABC Food Mart Brooklyn 3",
            "state": "NY",
            "city": "Brooklyn",
            "street": "Atlantic Avenue",
            "zip": "11217"
        }
    ]
    
    # Create a DataFrame
    df = pd.DataFrame(stores)
    return df

store_df = generate_store_data()

# Display the DataFrame
store_df

,store_id,store_name,state,city,street,zip
0,1,ABC Food Mart Queens 1,NY,Queens,Main Street,11354
1,2,ABC Food Mart Queens 2,NY,Queens,Broadway,11372
2,3,ABC Food Mart Brooklyn 1,NY,Brooklyn,Flatbush Avenue,11226
3,4,ABC Food Mart Brooklyn 2,NY,Brooklyn,Fulton Street,11201
4,5,ABC Food Mart Brooklyn 3,NY,Brooklyn,Atlantic Avenue,11217


In [11]:
# Define the table and column structure
table_name = "store"
columns = ", ".join(store_df.columns)
placeholders = ", ".join(["%s"] * len(store_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in store_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the store table.


### Employee

Managers can add employees as they are hired and onboarded using the emplyee admin portal

In [12]:
import random
from faker import Faker

# Initialize Faker
fake = Faker()

def generate_phone_number():
    """
    Generates a phone number with exactly 12 characters in the format 'XXX-XXX-XXXX'.
    Returns:
        str: A formatted phone number string.
    """
    return f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"

# Function to generate employee data
def generate_employee_data(num_employees_per_store=30, stores=None, departments=None, positions=None):
    """
    Generates employee data for multiple stores.

    Args:
        num_employees_per_store (int): Number of employees to generate for each store.
        stores (list): List of store data dictionaries.
        departments (list): List of department data dictionaries.
        positions (list): List of position data dictionaries.

    Returns:
        pd.DataFrame: A DataFrame containing employee data.
    """
    if stores is None or departments is None or positions is None:
        raise ValueError("Stores, departments, and positions data must be provided.")

    employee_data = []
    employee_id = 1  # Start employee ID

    for store in stores:
        for _ in range(num_employees_per_store):
            dept = random.choice(departments)
            position = random.choice(positions)
            employee = {
                "employee_id": employee_id,
                "store_id": store["store_id"],
                "dept_id": dept["dept_id"],
                "position_id": position["position_id"],
                "first_name": fake.first_name(),
                "last_name": fake.last_name(),
                "email": fake.email(),
                "phone_number": generate_phone_number(),
                "state": store["state"],
                "city": store["city"],
                "street": fake.street_name(),
                "zip": store["zip"],
                "hire_date": fake.date_between(start_date="-5y", end_date="today"),
            }
            employee_data.append(employee)
            employee_id += 1

    return pd.DataFrame(employee_data)

# Example store, department, and position data
stores = [
    {"store_id": 1, "state": "NY", "city": "Queens", "street": "Main Street", "zip": "11354"},
    {"store_id": 2, "state": "NY", "city": "Queens", "street": "Broadway", "zip": "11372"},
    {"store_id": 3, "state": "NY", "city": "Brooklyn", "street": "Flatbush Avenue", "zip": "11226"},
    {"store_id": 4, "state": "NY", "city": "Brooklyn", "street": "Fulton Street", "zip": "11201"},
    {"store_id": 5, "state": "NY", "city": "Brooklyn", "street": "Atlantic Avenue", "zip": "11217"}
]

departments = [{"dept_id": i, "dept_name": f"Department {i}"} for i in range(1, 11)]  # 10 Departments
positions = [{"position_id": i, "position_name": f"Position {i}"} for i in range(1, 13)]  # 12 Positions

# Generate employee data
employee_df = generate_employee_data(
    num_employees_per_store=30,
    stores=stores,
    departments=departments,
    positions=positions,
)

In [13]:
employee_df

,employee_id,store_id,dept_id,position_id,first_name,last_name,email,phone_number,state,city,street,zip,hire_date
0,1,1,4,7,Ronald,Thomas,travisramos@example.net,989-404-3919,NY,Queens,Alexandria Crossroad,11354,2022-07-14
1,2,1,9,2,Christopher,Burke,fraziertodd@example.com,877-200-4200,NY,Queens,Ali Drives,11354,2024-05-11
2,3,1,10,7,Emily,Long,drew13@example.com,300-566-5125,NY,Queens,Jones Square,11354,2023-09-06
3,4,1,5,12,Shawn,Alexander,smithbrandon@example.org,726-336-3325,NY,Queens,Owens Lake,11354,2024-06-07
4,5,1,3,7,Kathleen,Johnson,thall@example.com,430-892-9731,NY,Queens,Shelley Corners,11354,2021-04-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,146,5,10,12,Maria,Rodriguez,taylordonna@example.net,108-731-5354,NY,Brooklyn,Bass Groves,11217,2022-06-06
146,147,5,5,1,Ashley,Zimmerman,racheljones@example.com,239-881-9486,NY,Brooklyn,Eric Tunnel,11217,2024-04-10
147,148,5,3,5,Eric,Berry,toddwatson@example.org,559-676-2914,NY,Brooklyn,Glenn Turnpike,11217,2020-12-06
148,149,5,7,8,Michael,Spence,toni59@example.net,982-566-1597,NY,Brooklyn,Walker Track,11217,2020-08-26


In [14]:
# Define the table and column structure
table_name = "employee"
columns = ", ".join(employee_df.columns)
placeholders = ", ".join(["%s"] * len(employee_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in employee_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the employee table.


#### Employee Schedule

We would have a calander feature that would allow super users to create schedules for employees and update employees through an application. 

In [15]:
def generate_employee_schedules(employee_ids, num_days=30, historical=True):
    """
    Generates schedules for employees over a specified number of days.

    Args:
        employee_ids (list): List of employee IDs.
        num_days (int): Number of days to generate schedules for.
        historical (bool): If True, generates schedules for past dates; 
                           otherwise, generates for future dates.

    Returns:
        pd.DataFrame: A DataFrame containing employee schedules.
    """
    if not employee_ids:
        raise ValueError("Employee IDs must be provided.")

    schedules = []
    schedule_id = 1

    for employee_id in employee_ids:
        for day in range(num_days):
            # Generate a date based on historical or future requirement
            if historical:
                schedule_date = datetime.now() - timedelta(days=day + 1)
            else:
                schedule_date = datetime.now() + timedelta(days=day + 1)
            
            # Randomize shift start and duration
            shift_start = fake.date_time_between_dates(
                datetime_start=schedule_date.replace(hour=6, minute=0, second=0),
                datetime_end=schedule_date.replace(hour=14, minute=0, second=0)
            )
            shift_duration = timedelta(hours=random.randint(4, 8))  # Random shift of 4-8 hours
            shift_end = shift_start + shift_duration

            # Add the schedule entry
            schedules.append({
                "schedule_id": schedule_id,
                "employee_id": employee_id,
                "start_time": shift_start,
                "end_time": shift_end
            })
            schedule_id += 1

    return pd.DataFrame(schedules)

In [16]:
employee_id_list = list(range(1, 91))

In [17]:
historical_schedules = generate_employee_schedules(employee_id_list, num_days=30, historical=True)
print(historical_schedules)

      schedule_id  employee_id                 start_time  \
0               1            1 2024-12-11 06:51:00.403709   
1               2            1 2024-12-10 06:41:12.505867   
2               3            1 2024-12-09 11:44:21.959120   
3               4            1 2024-12-08 13:37:54.854136   
4               5            1 2024-12-07 08:37:53.891453   
...           ...          ...                        ...   
2695         2696           90 2024-11-16 08:49:51.934421   
2696         2697           90 2024-11-15 06:13:38.023682   
2697         2698           90 2024-11-14 11:18:49.268081   
2698         2699           90 2024-11-13 08:19:31.171140   
2699         2700           90 2024-11-12 13:31:39.274332   

                       end_time  
0    2024-12-11 13:51:00.403709  
1    2024-12-10 11:41:12.505867  
2    2024-12-09 16:44:21.959120  
3    2024-12-08 19:37:54.854136  
4    2024-12-07 14:37:53.891453  
...                         ...  
2695 2024-11-16 15:49:51.9344

##### historical employee schedules

In [18]:
# Define the table and column structure
table_name = "employee_schedule"
columns = ", ".join(historical_schedules.columns)
placeholders = ", ".join(["%s"] * len(historical_schedules.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in historical_schedules.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the employee_schedule table.


### Employee Time Tracking

A punch in and out feature of our employee app would allow for employees time working

In [19]:
# Function to generate historical time tracking data
def generate_time_tracking_data(employee_ids, num_days=30):
    """
    Generates historical time tracking data for employees.

    Args:
        employee_ids (list): List of employee IDs.
        num_days (int): Number of past days to generate data for.

    Returns:
        pd.DataFrame: A DataFrame containing employee time tracking data.
    """
    if not employee_ids:
        raise ValueError("Employee IDs must be provided.")

    time_tracking_data = []
    time_tracking_id = 1  # Start tracking ID (if needed for database insertion)

    for employee_id in employee_ids:
        for day in range(num_days):
            # Generate a historical date
            work_date = datetime.now() - timedelta(days=day + 1)
            
            # Randomize shift start and duration
            shift_start = fake.date_time_between_dates(
                datetime_start=work_date.replace(hour=6, minute=0, second=0),
                datetime_end=work_date.replace(hour=14, minute=0, second=0)
            )
            shift_duration = timedelta(hours=random.randint(4, 8))  # Random shift duration of 4-8 hours
            shift_end = shift_start + shift_duration
            
            # Add the time tracking entry
            time_tracking_data.append({
                "time_tracking_id": time_tracking_id,
                "employee_id": employee_id,
                "start_time": shift_start,
                "end_time": shift_end,
                "created_at": datetime.now(),
                "updated_at": datetime.now(),
            })
            time_tracking_id += 1

    return pd.DataFrame(time_tracking_data)

In [20]:
time_tracking_df = generate_time_tracking_data(employee_id_list, num_days=30)

time_tracking_df

,time_tracking_id,employee_id,start_time,end_time,created_at,updated_at
0,1,1,2024-12-11 11:41:38.274955,2024-12-11 16:41:38.274955,2024-12-12 11:34:37.388086,2024-12-12 11:34:37.388086
1,2,1,2024-12-10 12:17:09.822108,2024-12-10 18:17:09.822108,2024-12-12 11:34:37.388110,2024-12-12 11:34:37.388110
2,3,1,2024-12-09 07:33:53.060828,2024-12-09 15:33:53.060828,2024-12-12 11:34:37.388134,2024-12-12 11:34:37.388135
3,4,1,2024-12-08 10:26:52.895451,2024-12-08 15:26:52.895451,2024-12-12 11:34:37.388155,2024-12-12 11:34:37.388155
4,5,1,2024-12-07 06:41:12.647761,2024-12-07 10:41:12.647761,2024-12-12 11:34:37.388175,2024-12-12 11:34:37.388175
...,...,...,...,...,...,...
2695,2696,90,2024-11-16 08:28:43.554402,2024-11-16 13:28:43.554402,2024-12-12 11:34:37.451470,2024-12-12 11:34:37.451471
2696,2697,90,2024-11-15 08:21:24.854568,2024-11-15 14:21:24.854568,2024-12-12 11:34:37.451488,2024-12-12 11:34:37.451488
2697,2698,90,2024-11-14 08:51:27.315111,2024-11-14 12:51:27.315111,2024-12-12 11:34:37.451505,2024-12-12 11:34:37.451505
2698,2699,90,2024-11-13 06:39:39.553933,2024-11-13 13:39:39.553933,2024-12-12 11:34:37.451522,2024-12-12 11:34:37.451522


In [21]:
# Define the table and column structure
table_name = "employee_time_tracking"
columns = ", ".join(time_tracking_df.columns)
placeholders = ", ".join(["%s"] * len(time_tracking_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in time_tracking_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the employee_time_tracking table.


### Payroll

This data is accessible through the employee admin application and can change the employees wage and is hooked up to an external accounting system to pay employees correctly.

In [22]:
def generate_fake_payroll(employee_ids, num_pay_periods=12):
    """
    Generates fake payroll data for employees with consistent hourly wages.

    Args:
        employee_ids (list): List of employee IDs.
        num_pay_periods (int): Number of pay periods to generate for each employee.

    Returns:
        pd.DataFrame: A DataFrame containing payroll data.
    """
    if not employee_ids:
        raise ValueError("Employee IDs must be provided.")

    payroll_data = []
    payroll_id = 1  # Start payroll ID (if needed for database insertion)

    # Assign consistent hourly wage to each employee
    employee_wages = {
        employee_id: round(random.uniform(15.00, 50.00), 2)
        for employee_id in employee_ids
    }

    for employee_id in employee_ids:
        for period in range(num_pay_periods):
            # Generate pay period start and end dates (bi-weekly schedule)
            pay_period_end = datetime.now() - timedelta(weeks=2 * period)
            pay_period_start = pay_period_end - timedelta(days=13)  # Bi-weekly pay period (14 days)

            # Retrieve consistent hourly wage for the employee
            hourly_wage = employee_wages[employee_id]

            # Add payroll entry
            payroll_data.append({
                "payroll_id": payroll_id,
                "employee_id": employee_id,
                "pay_period_start": pay_period_start.date(),
                "pay_period_end": pay_period_end.date(),
                "hour_wage": hourly_wage,
            })
            payroll_id += 1

    return pd.DataFrame(payroll_data)

In [23]:
payroll_df = generate_fake_payroll(employee_id_list, num_pay_periods=2)

payroll_df

,payroll_id,employee_id,pay_period_start,pay_period_end,hour_wage
0,1,1,2024-11-29,2024-12-12,40.88
1,2,1,2024-11-15,2024-11-28,40.88
2,3,2,2024-11-29,2024-12-12,36.04
3,4,2,2024-11-15,2024-11-28,36.04
4,5,3,2024-11-29,2024-12-12,44.81
...,...,...,...,...,...
175,176,88,2024-11-15,2024-11-28,36.14
176,177,89,2024-11-29,2024-12-12,21.51
177,178,89,2024-11-15,2024-11-28,21.51
178,179,90,2024-11-29,2024-12-12,42.40


In [24]:
# Define the table and column structure
table_name = "payroll"
columns = ", ".join(payroll_df.columns)
placeholders = ", ".join(["%s"] * len(payroll_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in payroll_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the payroll table.


### Vendors

Store managers can update vendors and there information through our super user at the application level

In [25]:
def generate_vendors(num_vendors=10):
    """
    Generates vendor data for a grocery store.

    Args:
        num_vendors (int): Number of vendors to generate.

    Returns:
        pd.DataFrame: A DataFrame containing vendor data.
    """
    vendors = []

    for vendor_id in range(1, num_vendors + 1):  # Start vendor_id from 1
        vendor = {
            "vendor_id": vendor_id,  # Assign vendor_id
            "vendor_name": fake.company()[:20],  # Truncate to 20 characters for SQL limit
            "phone_number": f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}",
            "email": fake.company_email(),
            "website": fake.url(),
        }
        vendors.append(vendor)

    # Ensure at least one contact method is present (per the SQL constraint)
    for vendor in vendors:
        if random.choice([True, False]):
            vendor["phone_number"] = None
        if not vendor["phone_number"] and random.choice([True, False]):
            vendor["email"] = None
        if not vendor["phone_number"] and not vendor["email"]:
            vendor["website"] = None

    return pd.DataFrame(vendors)

In [26]:
# Generate vendor data
vendors_df = generate_vendors(10)
vendors_df.iloc(1)

In [27]:
vendors_df

,vendor_id,vendor_name,phone_number,email,website
0,1,Everett Ltd,None,None,None
1,2,"Newman, Cabrera and",788-391-5446,byrdjennifer@rivera.com,http://oconnor-scott.biz/
2,3,Mckay-Hughes,None,laura19@wood.com,https://www.rivera.com/
3,4,Crawford-Lopez,None,None,None
4,5,Nguyen-Moore,None,todd87@roman.com,http://www.wright.org/
5,6,Reynolds LLC,443-215-1411,pamelacunningham@booker.org,http://www.wells.com/
6,7,"Martinez, Griffin an",313-817-2501,qwalker@page.info,http://www.ellis.com/
7,8,Clark-Wilcox,321-874-3271,gdean@bishop.org,http://ortiz.com/
8,9,Russell-Bradley,None,None,None
9,10,"Parker, Richardson a",889-937-8245,xbecker@walker.com,https://www.washington-jenkins.org/


In [28]:
# Define the table and column structure
table_name = "vendors"
columns = ", ".join(vendors_df.columns)
placeholders = ", ".join(["%s"] * len(vendors_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in vendors_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the vendors table.


### Products

Products can easily be updated by store managers when new products arrive and can be scanned with a barcode. This can create new product ids and fill in missing product information. Each product is tied to a vendor to help ensure that poor products or spoiled items are noted easily from the products orgin.

In [29]:
def generate_grocery_products(num_products=100):
    """
    Generates a DataFrame of unique grocery store products.
    
    Args:
        num_products (int): Number of products to generate. Default is 100.
    
    Returns:
        pd.DataFrame: A DataFrame containing product_id, dept_id, vendor_id, brand, and product_name.
    """
    # Departments and example product categories associated with each department
    department_products = {
        1: ["Apples", "Bananas", "Oranges", "Grapes", "Strawberries"],
        2: ["Whole Milk", "Cheddar Cheese", "Greek Yogurt", "Butter", "Cream Cheese"],
        3: ["White Bread", "Baguette", "Croissant", "Muffin", "Donut"],
        4: ["Frozen Pizza", "Ice Cream", "Frozen Vegetables", "Frozen Fries", "Frozen Berries"],
        5: ["Chicken Breast", "Salmon Fillet", "Ground Beef", "Pork Chops", "Shrimp"],
        6: ["Spaghetti Pasta", "Brown Rice", "Canned Beans", "Peanut Butter", "Olive Oil"],
        7: ["Orange Juice", "Bottled Water", "Cola", "Green Tea", "Sparkling Water"],
        8: ["Paper Towels", "Toilet Paper", "Dish Soap", "Laundry Detergent", "Trash Bags"],
        9: ["Shampoo", "Toothpaste", "Deodorant", "Hand Soap", "Body Lotion"],
        10: ["Potato Chips", "Chocolate Bar", "Granola Bars", "Pretzels", "Popcorn"]
    }

    # Example brands
    brands = [
        "FreshFarm", "GoodFood", "OrganicChoice", "BudgetBuy", "NatureDelight",
        "HarvestBest", "DailyEssentials", "GreenValley", "PureTaste", "FamilyFare"
    ]

    # Vendors (1 to 10)
    vendors = list(range(1, 11))

    # Ensure uniqueness by tracking used (brand, product_name) pairs
    used_combinations = set()

    products = []
    product_id = 1

    while len(products) < num_products:
        dept_id = random.choice(list(department_products.keys()))
        vendor_id = random.choice(vendors)
        brand = random.choice(brands)
        product_name = random.choice(department_products[dept_id])

        # Combine brand and product name to form a unique product descriptor
        full_product_name = f"{brand} {product_name}"
        
        # Ensure uniqueness
        if full_product_name not in used_combinations:
            used_combinations.add(full_product_name)
            
            products.append({
                "product_id": product_id,
                "dept_id": dept_id,
                "vendor_id": vendor_id,
                "brand": brand,
                "product_name": full_product_name
            })
            product_id += 1

    return pd.DataFrame(products)

In [30]:
products_df = generate_grocery_products(100)
products_df

,product_id,dept_id,vendor_id,brand,product_name
0,1,5,5,DailyEssentials,DailyEssentials Ground Beef
1,2,10,10,PureTaste,PureTaste Potato Chips
2,3,3,10,HarvestBest,HarvestBest Muffin
3,4,9,3,PureTaste,PureTaste Toothpaste
4,5,2,1,BudgetBuy,BudgetBuy Whole Milk
...,...,...,...,...,...
95,96,10,10,OrganicChoice,OrganicChoice Granola Bars
96,97,8,9,DailyEssentials,DailyEssentials Toilet Paper
97,98,4,3,GoodFood,GoodFood Frozen Vegetables
98,99,7,2,DailyEssentials,DailyEssentials Green Tea


In [31]:
# Define the table and column structure
table_name = "products"
columns = ", ".join(products_df.columns)
placeholders = ", ".join(["%s"] * len(products_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in products_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the products table.


#### Price History

Store managers and owners can update pricing of products using this table. On an application level a user would be able to first find a specific product and adjust the price from an effective start date. If no new product price is implemented the most recent product price is used.

In [32]:
def generate_price_history_with_timestamps(products_df, start_date="2023-10-01", num_stores=5):
    """
    Generates price history for products from a given start date to the present with timestamps.
    The 'end_date' column has been removed from the output.

    Args:
        products_df (pd.DataFrame): DataFrame containing product data.
        start_date (str): The start date for price history generation (format: YYYY-MM-DD).
        num_stores (int): Number of stores associated with each product.

    Returns:
        pd.DataFrame: A DataFrame containing price history records with timestamps.
    """
    # Convert start_date to a datetime object
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.now()
    price_history = []

    # Iterate through each product
    for _, product in products_df.iterrows():
        product_id = product["product_id"]
        
        # Generate price history for each store
        for store_id in range(1, num_stores + 1):
            current_date = start_date
            while current_date < end_date:
                price = round(random.uniform(1.00, 50.00), 2)  # Random price between $1.00 and $50.00
                
                # Calculate effective_date with timestamps
                effective_date = current_date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59))

                # Append the price history record
                price_history.append({
                    "product_id": product_id,
                    "store_id": store_id,
                    "price": price,
                    "effective_date": effective_date
                })

                # Update the current date for the next price period
                current_date = effective_date + timedelta(days=random.randint(7, 14))  # Price valid for 1-2 weeks

    return pd.DataFrame(price_history)

In [33]:
# Generate price history with timestamps
price_history_df = generate_price_history_with_timestamps(products_df)

In [34]:
price_history_df.tail(10)

,product_id,store_id,price,effective_date
20166,100,5,44.36,2024-08-21 14:00:00
20167,100,5,46.02,2024-08-28 20:20:00
20168,100,5,35.48,2024-09-09 13:46:00
20169,100,5,37.26,2024-09-23 12:48:00
20170,100,5,14.64,2024-10-05 08:09:00
20171,100,5,47.35,2024-10-19 16:44:00
20172,100,5,6.29,2024-11-02 04:15:00
20173,100,5,37.93,2024-11-10 18:10:00
20174,100,5,26.80,2024-11-19 19:13:00
20175,100,5,16.76,2024-12-03 17:53:00


In [35]:
# Define the table and column structure
table_name = "price_history"
columns = ", ".join(price_history_df.columns)
placeholders = ", ".join(["%s"] * len(price_history_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in price_history_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the price_history table.


### Deliveries

Deliveries are an important part of the inventory system. Each delivery must be made by a known vendor and may contain multiple products at varying quantities. As deliveries are created from vendors a system on the application level allows for first a delivery notice to the vendor. As delivery items are confirmed and a date is set a delivery is created through our website. When the delivery is delivered the delivery can be scanned and all contents with it. This would be done through a mobile app to allow inventory managers to keep accurate records.

In [36]:
def generate_deliveries_data(num_deliveries=1000, vendors=range(1, 11), products=range(1, 101), stores=range(1, 6)):
    """
    Generates data for deliveries to populate the deliveries table.

    Args:
        num_deliveries (int): Number of delivery records to generate.
        vendors (range or list): Range or list of vendor IDs (default: 3 to 12 inclusive).
        products (range or list): Range or list of product IDs (default: 1 to 100 inclusive).
        stores (range or list): Range or list of store IDs (default: 1 to 3 inclusive).

    Returns:
        pd.DataFrame: A DataFrame containing delivery data.
    """
    deliveries = []
    
    for _ in range(num_deliveries):
        vendor_id = random.choice(vendors)
        product_id = random.choice(products)
        store_id = random.choice(stores)
        delivery_date = datetime.now() - timedelta(days=random.randint(1, 180))  # Random date in the past 6 months
        quantity = random.randint(100, 1000)  # Random quantity between 100 and 1000

        deliveries.append({
            "vendor_id": vendor_id,
            "product_id": product_id,
            "store_id": store_id,
            "delivery_date": delivery_date.date(),
            "quantity": quantity
        })
    
    return pd.DataFrame(deliveries)

In [37]:
# Generate delivery data
deliveries_df = generate_deliveries_data(num_deliveries=1000)

# Display the DataFrame
deliveries_df.head(10)

,vendor_id,product_id,store_id,delivery_date,quantity
0,1,95,2,2024-06-23,919
1,6,39,5,2024-12-10,370
2,8,81,1,2024-10-03,528
3,6,34,1,2024-10-09,533
4,8,41,2,2024-10-23,424
5,4,1,4,2024-07-14,669
6,2,25,4,2024-07-24,118
7,10,20,2,2024-10-08,178
8,7,53,4,2024-11-11,881
9,4,43,1,2024-11-26,902


In [38]:
# Define the table and column structure
table_name = "deliveries"
columns = ", ".join(deliveries_df.columns)
placeholders = ", ".join(["%s"] * len(deliveries_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in deliveries_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the deliveries table.


A trigger is also set that when a delivery is executed the inventory table is updated.

### Sale and Transaction

The ETL comes from the point of sales system transaction id and les tables are used to keep track of important audit information. This means keeping our own records of each transaction and sales of items. This also automatically updates inventory at each store to allow store managers to make new orders on products that are low in stock.

In [39]:
def generate_transactions_and_sales(num_transactions=5000):
    """
    Generates transactions and sales data starting from October 1, 2023.

    Args:
        num_transactions (int): Number of transactions to generate.

    Returns:
        tuple: Two DataFrames, one for transactions and one for sales.
    """
    transactions = []
    sales = []

    transaction_id = 1
    start_date = datetime(2024, 10, 1)  # Start date for transactions
    end_date = datetime.now()  # Current date as the end date

    for _ in range(num_transactions):
        # Randomize store_id, transaction_type, and transaction_date
        store_id = random.randint(1, 5)
        transaction_type = 0 if random.random() < 0.95 else 1  # 95% sales, 5% returns
        transaction_date = start_date + timedelta(
            days=random.randint(0, (end_date - start_date).days),  # Random day within range
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59)
        )

        # Add transaction record
        transactions.append({
            "transaction_id": transaction_id,
            "store_id": store_id,
            "transaction_date": transaction_date,
            "transaction_type": transaction_type
        })

        # Generate a random number of sales/returns for this transaction
        num_items = random.randint(1, 5)  # Each transaction involves 1 to 5 items
        for _ in range(num_items):
            product_id = random.randint(1, 100)  # Random product ID
            quantity = random.randint(1, 5)  # Random quantity (1 to 10)

            # Add sale record (quantity should be positive, even for returns)
            sales.append({
                "transaction_id": transaction_id,
                "product_id": product_id,
                "quantity": quantity
            })

        transaction_id += 1

    # Convert to DataFrames
    transactions_df = pd.DataFrame(transactions)
    sales_df = pd.DataFrame(sales)

    return transactions_df, sales_df


In [40]:
# Generate transactions and sales data
transactions_df, sales_df = generate_transactions_and_sales(500)

# Display samples
print("Transactions Sample:")
print(transactions_df.head())

print("\nSales Sample:")
print(sales_df.head())

Transactions Sample:
   transaction_id  store_id    transaction_date  transaction_type
0               1         1 2024-11-07 01:15:00                 0
1               2         4 2024-11-25 20:58:00                 0
2               3         4 2024-11-16 05:57:00                 0
3               4         3 2024-10-14 08:40:00                 0
4               5         5 2024-11-05 20:12:00                 0

Sales Sample:
   transaction_id  product_id  quantity
0               1          87         1
1               2          49         5
2               2          87         1
3               2          80         1
4               2          48         3


In [41]:
# Define the table and column structure
table_name = "transactions"
columns = ", ".join(transactions_df.columns)
placeholders = ", ".join(["%s"] * len(transactions_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in transactions_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the transactions table.


In [42]:
# Define the table and column structure
table_name = "sales"
columns = ", ".join(sales_df.columns)
placeholders = ", ".join(["%s"] * len(sales_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in sales_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the sales table.


### Customer Information

Customers can create accounts with abc_mart and will be added to an email list.

In [43]:
def generate_customers(num_customers=1000):
    """
    Generates customer data for populating the customers table.

    Args:
        num_customers (int): Number of customers to generate.

    Returns:
        pd.DataFrame: A DataFrame containing customer data.
    """
    fake = Faker()
    customers = []

    for _ in range(num_customers):
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = fake.unique.email()
        phone_number = f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        date_joined = datetime.now() - timedelta(days=random.randint(0, 365 * 2))  # Random date in past 2 years

        customers.append({
            "first_name": first_name,
            "last_name": last_name,
            "email": email,
            "phone_number": phone_number,
            "date_joined": date_joined
        })

    return pd.DataFrame(customers)


In [44]:
# Generate customer data
customers_df = generate_customers(1000)

customers_df.head()

,first_name,last_name,email,phone_number,date_joined
0,Ronald,Torres,curtisanthony@example.org,469-922-9431,2023-04-14 11:34:39.192594
1,Crystal,Shelton,caitlinsmith@example.org,227-427-6057,2024-01-24 11:34:39.192769
2,Tiffany,Graham,heathersandoval@example.net,636-907-7607,2023-05-07 11:34:39.192977
3,Elizabeth,Hogan,cwest@example.com,616-709-1705,2023-11-30 11:34:39.193082
4,Kellie,Lopez,adam19@example.com,447-850-1480,2023-04-04 11:34:39.193171


In [45]:
# Define the table and column structure
table_name = "customers"
columns = ", ".join(customers_df.columns)
placeholders = ", ".join(["%s"] * len(customers_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in customers_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the customers table.


### Customer Reviews

##### In customer reviews while leaving a review on the website, customers must be identified so they can be added to mailing list. If a space is left blank it is defaulted to 0 and disregarded in analysis.

##### As part of the ETL process 0 are placeholders in both product and store tables to allow customers to leave reviews naturally about the product, store, or both. All reviews are stored in this table. Customers can review a store, product, or both.

In [46]:
def add_placeholders():
    try:
        cur.execute("INSERT INTO products (product_id, product_name) VALUES (0, 'No Product') ON CONFLICT DO NOTHING;")
        cur.execute("INSERT INTO store (store_id, store_name) VALUES (0, 'No Store') ON CONFLICT DO NOTHING;")
        conn.commit()
        print("Placeholders added successfully.")
    except Exception as e:
        conn.rollback()
        print(f"Error adding placeholders: {e}")

In [47]:
# Generate customer reviews
def generate_customer_reviews(num_reviews=100, customer_ids=range(1, 101), store_ids=range(0, 6), product_ids=range(0, 101)):
    """
    Generates customer reviews for populating the customer_reviews table.

    Args:
        num_reviews (int): Number of reviews to generate.
        customer_ids (range or list): List of customer IDs.
        store_ids (range or list): List of store IDs (includes 0 for 'No Store').
        product_ids (range or list): List of product IDs (includes 0 for 'No Product').

    Returns:
        pd.DataFrame: A DataFrame containing customer review data.
    """
    fake = Faker()
    reviews = []

    for _ in range(num_reviews):
        customer_id = random.choice(customer_ids)
        store_id = random.choice(store_ids) if random.random() < 0.7 else 0  # 70% chance of a store review
        product_id = random.choice(product_ids) if store_id == 0 or random.random() < 0.7 else 0  # 70% chance of a product review
        rating = random.randint(1, 5)  # Random rating between 1 and 5
        review_text = fake.sentence(nb_words=random.randint(5, 20)) if random.random() < 0.8 else ""  # 80% chance of review text
        review_date = datetime.now() - timedelta(days=random.randint(1, 365))  # Random date in past year

        # Ensure at least one of store_id or product_id is not 0
        if store_id == 0 and product_id == 0:
            store_id = random.choice(store_ids)

        reviews.append({
            "customer_id": customer_id,
            "store_id": store_id,
            "product_id": product_id,
            "rating": rating,
            "review_text": review_text,
            "review_date": review_date
        })

    # Create a DataFrame
    reviews_df = pd.DataFrame(reviews)

    return reviews_df

In [48]:
# Generate reviews
customer_reviews_df = generate_customer_reviews(200)

customer_reviews_df.head(10)

,customer_id,store_id,product_id,rating,review_text,review_date
0,98,4,75,5,,2024-10-29 11:34:39.386412
1,10,3,42,4,Yes more hold question professor address fight...,2024-02-17 11:34:39.386681
2,33,3,60,5,Man range understand edge great least turn hot...,2024-01-21 11:34:39.386697
3,73,5,0,1,Against full business summer out case build ca...,2024-12-03 11:34:39.386709
4,8,0,16,5,Doctor big interest its doctor later attorney ...,2024-10-19 11:34:39.386720
5,97,4,43,4,Grow me training late until point major site m...,2024-04-10 11:34:39.386731
6,5,2,37,2,Relate bed learn personal question shake middl...,2024-10-04 11:34:39.386742
7,85,5,0,3,Lose wide moment better house.,2024-07-06 11:34:39.386754
8,32,0,62,3,Left court check nation appear kid specific to...,2024-06-13 11:34:39.386768
9,18,0,3,4,,2024-07-05 11:34:39.386770


In [49]:
# Step 3: Insert customer reviews into the database
def insert_customer_reviews(reviews_df):
    try:
        table_name = "customer_reviews"
        columns = ", ".join(reviews_df.columns)
        placeholders = ", ".join(["%s"] * len(reviews_df.columns))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        for row in reviews_df.itertuples(index=False, name=None):
            cur.execute(insert_query, row)

        conn.commit()
        print("Customer reviews inserted successfully.")
    except Exception as e:
        conn.rollback()
        print(f"Error inserting customer reviews: {e}")

# Execute the steps
if __name__ == "__main__":
    add_placeholders()
    customer_reviews_df = generate_customer_reviews(num_reviews=100)
    insert_customer_reviews(customer_reviews_df)

Placeholders added successfully.
Customer reviews inserted successfully.


### Product Waste

#### Through an employee app managers can track product waste and also update inventory as necassary.

In [50]:
def fetch_inventory_levels(store_id, product_id):
    """
    Fetch the available inventory for a specific store and product.

    Args:
        store_id (int): The ID of the store.
        product_id (int): The ID of the product.

    Returns:
        int: Available inventory quantity. Defaults to None if no record exists.
    """
    cur.execute("SELECT quantity FROM inventory WHERE store_id = %s AND product_id = %s", (store_id, product_id))
    result = cur.fetchone()
    return result[0] if result else None

In [51]:
def generate_food_waste(num_records=50, product_ids=range(1, 101), store_ids=range(1, 6), dept_ids=range(1, 11)):
    """
    Generates instances of food waste data for the product_waste table.

    Args:
        num_records (int): Number of food waste records to generate.
        product_ids (range or list): List of product IDs.
        store_ids (range or list): List of store IDs.
        dept_ids (range or list): List of department IDs.

    Returns:
        pd.DataFrame: A DataFrame containing food waste data.
    """
    fake = Faker()
    waste_data = []

    for _ in range(num_records):
        product_id = random.choice(product_ids)
        store_id = random.choice(store_ids)
        dept_id = random.choice(dept_ids)

        # Fetch the available inventory
        available_inventory = fetch_inventory_levels(store_id, product_id)

        # Skip this entry if no inventory exists or if inventory is 0
        if available_inventory is None or available_inventory <= 0:
            print(f"Skipping store_id={store_id}, product_id={product_id}: No available inventory.")
            continue

        # Generate a waste quantity that does not exceed available inventory
        quantity = random.randint(1, min(available_inventory, 50))
        waste_reason = random.choice([
            "Expired Product",
            "Damaged Packaging",
            "Spoiled",
            "Customer Return",
            "Incorrect Delivery"
        ])
        waste_date = datetime.now() - timedelta(days=random.randint(1, 180))

        waste_data.append({
            "product_id": product_id,
            "store_id": store_id,
            "dept_id": dept_id,
            "quantity": quantity,
            "waste_reason": waste_reason,
            "waste_date": waste_date
        })

    return pd.DataFrame(waste_data)


In [61]:
food_waste_df = generate_food_waste(50)
food_waste_df.head(15)

Skipping store_id=2, product_id=66: No available inventory.
Skipping store_id=3, product_id=97: No available inventory.
Skipping store_id=2, product_id=9: No available inventory.
Skipping store_id=5, product_id=97: No available inventory.


,product_id,store_id,dept_id,quantity,waste_reason,waste_date
0,15,3,6,19,Customer Return,2024-09-21 11:35:28.503409
1,15,5,6,38,Damaged Packaging,2024-08-24 11:35:28.503643
2,64,5,3,6,Damaged Packaging,2024-11-29 11:35:28.503803
3,95,3,2,26,Customer Return,2024-10-07 11:35:28.503989
4,53,5,10,13,Customer Return,2024-11-22 11:35:28.504190
5,82,5,2,28,Spoiled,2024-09-08 11:35:28.504385
6,48,1,4,34,Damaged Packaging,2024-12-02 11:35:28.504804
7,18,1,5,1,Damaged Packaging,2024-07-29 11:35:28.504965
8,22,5,2,33,Incorrect Delivery,2024-07-20 11:35:28.505118
9,25,5,4,41,Customer Return,2024-08-13 11:35:28.505266


In [53]:
# Define the table and column structure
table_name = "product_waste"
columns = ", ".join(food_waste_df.columns)
placeholders = ", ".join(["%s"] * len(food_waste_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in food_waste_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the product_waste table.


In [54]:
# Close the cursor
cur.close()

# Close the connection
conn.close()

print("Database connection closed.")

Database connection closed.
